### R中使用KNN算法

首先是使用`class`库调用普通的ＫＮＮ算法，之后使用｀ｍｅａｎＳｈｉｆｔＲ｀调用KNN的｀KD-Tree｀实现。

#### ＫＮＮ

参考[knn function](https://www.rdocumentation.org/packages/class/versions/7.3-15/topics/knn)

调用方式：
`knn(train, test, cl, k = 1, l = 0, prob = FALSE, use.all = TRUE)`

```
Arguments

train： matrix or data frame of training set cases.

test：　matrix or data frame of test set cases. A vector will be interpreted as a row vector for a single case.

cl：　factor of true classifications of training set

k：　number of neighbours considered.

l：　minimum vote for definite decision, otherwise doubt. (More precisely, less than k-l dissenting votes are allowed, even if k is increased by ties.)

prob：　If this is true, the proportion of the votes for the winning class are returned as attribute prob.

use.all：　controls handling of ties. If true, all distances equal to the kth largest are included. If false, a random selection of distances equal to the kth is chosen to use exactly k neighbours.

```

In [20]:
library(class)
set.seed(123)
in_train <- sample(1:nrow(iris), size = 100)
train_data <- iris[in_train,-5]
train_label <- iris[in_train, 5]
test_data  <- iris[-in_train, -5]
test_label  <- iris[-in_train, 5]
cl <- factor(iris[in_train, 5])
knn.pred <- knn(train_data, test_data, cl, k = 3)

In [21]:
# 混淆矩阵
table(knn.pred, test_label)

            test_label
knn.pred     setosa versicolor virginica
  setosa         14          0         0
  versicolor      0         17         1
  virginica       0          2        16

In [3]:
# 测试集上的准确率
test_size <- nrow(test_data)
(14+17+16)/test_size

[1] 0.94

In [4]:
knn.pred

[1] setosa     setosa     setosa     setosa     setosa     setosa    
 [7] setosa     setosa     setosa     setosa     setosa     setosa    
[13] setosa     setosa     versicolor versicolor versicolor versicolor
[19] versicolor versicolor versicolor versicolor virginica  versicolor
[25] versicolor versicolor versicolor virginica  versicolor versicolor
[31] versicolor versicolor versicolor virginica  virginica  virginica 
[37] virginica  versicolor virginica  virginica  virginica  virginica 
[43] virginica  virginica  virginica  virginica  virginica  virginica 
[49] virginica  virginica 
Levels: setosa versicolor virginica

#### KNN(KD-Tree)

从[meanShiftR](https://github.com/jlisic/meanShiftR)库调用函数`knn_meanShift`：
>knn_meanShift performs a search for the k nearest neighbors of a single point, where nearest isdetermined by the Mahalanobis distance. This search is performed through a k-d tree.

>此外，[RANN](https://github.com/jefferis/RANN)库也有类似的实现。

调用方式：
`knn_meanShift(points, trainData, k = min(5, NROW(trainData)), weight,leafSize = 40, maxDist = Inf）`

```
Arguments：

points：　n vectors stored in an n by p matrix.  k nearest neighbors are found for each　vector.

trainData：　A matrix or vector of potential nearest neighbors.

k： A scalar indicating the number neighbors to find.

weight：　A scalar or vector of length equal to the number of columns of　trainData. This　value is used as the diagonal elements for the inverse covariance matrix of the　Mahalanobis distance.

leafSize： A scalar used to specify the number of points to store in the leaf nodes.

maxDist： A vector specifying the maximum value of the Mahalanobis that will be considered

```

In [26]:
library(meanShiftR)
neigh <- knn_meanShift(test_data, train_data, k = 3)
neigh$neighbors

40,42,60
42,35,48
72,40,60
35,60,42
45,90,18
90,77,49
30,82,56
94,35,6
60,48,54
42,40,60
75,49,77


In [27]:
# 根据case index返回case class
# 根据neighbor cases的index获取预测的最终类别
getclass <- function(indexes){
    v <- sapply(indexes, function(index) train_label[index])
    uniqv <- unique(v)
    uniqv[which.max(tabulate(match(v, uniqv)))]
}
test_pred <- apply(neigh$neighbors, 1, getclass)

In [28]:
# 混淆矩阵
table(test_pred, iris[-in_train, 5])

            
test_pred    setosa versicolor virginica
  setosa         14          0         0
  versicolor      0         17         1
  virginica       0          2        16

In [29]:
test_pred

[1] setosa     setosa     setosa     setosa     setosa     setosa    
 [7] setosa     setosa     setosa     setosa     setosa     setosa    
[13] setosa     setosa     versicolor versicolor versicolor versicolor
[19] versicolor versicolor versicolor versicolor virginica  versicolor
[25] versicolor versicolor versicolor virginica  versicolor versicolor
[31] versicolor versicolor versicolor virginica  virginica  virginica 
[37] virginica  versicolor virginica  virginica  virginica  virginica 
[43] virginica  virginica  virginica  virginica  virginica  virginica 
[49] virginica  virginica 
Levels: setosa versicolor virginica